In [8]:
import pandas as pd
from datetime import datetime

df = pd.read_excel("/content/rfm_data_1.xlsx")


In [9]:
df.head()

,TransactionDate,TransactionAmount,CustomerID
0,2010-12-01 08:26:00,2.55,17850.0
1,2010-12-01 08:26:00,3.39,17850.0
2,2010-12-01 08:26:00,2.75,17850.0
3,2010-12-01 08:26:00,3.39,17850.0
4,2010-12-01 08:26:00,3.39,17850.0


In [10]:
df.CustomerID.nunique()

4372

In [11]:
df.shape

(541910, 3)

In [ ]:
df.info()

In [12]:
df.describe()

,TransactionDate,TransactionAmount,CustomerID
count,541910,541910.000000,406830.000000
mean,2011-07-04 13:35:22.342307584,4.611138,15287.684160
min,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,2011-03-28 11:34:00,1.250000,13953.000000
50%,2011-07-19 17:17:00,2.080000,15152.000000
75%,2011-10-19 11:27:00,4.130000,16791.000000
max,2011-12-09 12:50:00,38970.000000,18287.000000
std,NaN,96.759765,1713.603074


In [13]:
df["TransactionDate"]=pd.to_datetime(df["TransactionDate"])

In [14]:
today = datetime(2024,7,12)
rfm= df.groupby('CustomerID').agg({
    'TransactionDate': lambda x: (today - x.max()).days,
    'CustomerID': 'count',
    'TransactionAmount':'sum'
})

rfm.rename(columns={
    'TransactionDate': 'Recency',
    'CustomerID': 'Frequency',
    'TransactionAmount': 'Monetary'
}, inplace=True)

print(rfm.head())

            Recency  Frequency  Monetary
CustomerID                              
12346.0        4923          2      2.08
12347.0        4600        182    481.21
12348.0        4673         31    178.71
12349.0        4616         73    605.10
12350.0        4908         17     65.30


In [15]:
quantiles = rfm.quantile(q=[0.25, 0.5, 0.75])

def rfm_segment(row):
    r_score = 1 if row['Recency'] <= quantiles['Recency'][0.25] else 2 if row['Recency'] <= quantiles['Recency'][0.5] else 3 if row['Recency'] <= quantiles['Recency'][0.75] else 4
    f_score = 4 if row['Frequency'] <= quantiles['Frequency'][0.25] else 3 if row['Frequency'] <= quantiles['Frequency'][0.5] else 2 if row['Frequency'] <= quantiles['Frequency'][0.75] else 1
    m_score = 4 if row['Monetary'] <= quantiles['Monetary'][0.25] else 3 if row['Monetary'] <= quantiles['Monetary'][0.5] else 2 if row['Monetary'] <= quantiles['Monetary'][0.75] else 1
    return str(r_score) + str(f_score) + str(m_score)

rfm['RFM_Segment'] = rfm.apply(rfm_segment, axis=1)
print(rfm.head())

            Recency  Frequency  Monetary RFM_Segment
CustomerID                                          
12346.0        4923          2      2.08         444
12347.0        4600        182    481.21         111
12348.0        4673         31    178.71         332
12349.0        4616         73    605.10         221
12350.0        4908         17     65.30         443
